# Extracting the data

In [28]:
import numpy as np
from sklearn import preprocessing
import tensorflow as tf

raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter = ',')

unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

# Shuffle before balancing the dataset

In [7]:
# When the data was collected it was actually arranged by date
# Shuffle the indices of the data, so the data is not arranged in any way when we feed it.
# Since we will be batching, we want the data to be as randomly spread out as possible
shuffled_indices = np.arange(unscaled_inputs_all.shape[0])
np.random.shuffle(shuffled_indices)

# Use the shuffled indices to shuffle the inputs and targets.
unscaled_inputs_all = unscaled_inputs_all[shuffled_indices]
targets_all = targets_all[shuffled_indices]

# Balance the Dataset


In [8]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
            
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis = 0)
targets_equal_priors = np.delete (targets_all, indices_to_remove, axis=0)

# Standardize the inputs

In [9]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

# Shuffle the data

In [10]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

# Split the dataset into train, validation and test

In [11]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1776.0 3579 0.49622799664710815
239.0 447 0.5346756152125279
222.0 448 0.4955357142857143


# Save the three datasets in *.npz

In [12]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

### Data

Loading the dataset, saving into variables

In [14]:
# Loading Train, Validation and Test inputs and targets respectively, saving them into temporary variable 'npz'
# Then assigning into input and target variables. Moreover, we are changing inputs datatypes to float and targets to integer.
npz = np.load('Audiobooks_data_train.npz')

train_inputs, train_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

npz = np.load('Audiobooks_data_validation.npz')

validation_inputs, validation_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

npz = np.load('Audiobooks_data_test.npz')

test_inputs, test_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

### Model
Outline, optimizers, loss, early stopping and training

In [19]:
# We've got 10 different predictors, therefore:
input_size = 10
# We've got 2 outputs: Whether customer will convert or not, thus output size is equal to 2
output_size = 2
# Hidden layer size, we can set to 50
hidden_layer_size = 50

# Now we build our model similarly to MNIST dataset, with a small difference: there's no need to flatten the tensor
# As we have already done the preprocessing here.
# Similarly to MNIST, we will build 2 hidden layers and use the same activation functions. Softmax will work splendidly here.
model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), #1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), #2nd hidden layer
    tf.keras.layers.Dense(output_size, activation='softmax') #Output layer
])

# Now, we shall choose the otpimizer and loss function: 
## We define optimizer
## Then loss function
## and the metric we are interested in obtaining

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training: time to train our model

## Let's set the batch size
batch_size = 100

## Set number of epochs
max_epochs = 100

## Setting the early stopping mechanism so our model won't overfit
### patience=2 indicates that our callback will let 2 validation accuracy increases to slide, before stopping the training.
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

# Fit the model
## We should note that train, validation and test data are not iterable here
model.fit(train_inputs, # train inputs
          train_targets, # train targets
          batch_size=batch_size, # batch size
          epochs=max_epochs, # n of epochs we will train for, assuming early stopping willn't occur
          callbacks=[early_stopping], # implementing the early stopping mechanism
          validation_data=(validation_inputs, validation_targets), # validation data
          verbose=2) # Making sure we get enough information about the training process after each epoch. 

Epoch 1/100
36/36 - 1s - loss: 0.6377 - accuracy: 0.6530 - val_loss: 0.5732 - val_accuracy: 0.7025 - 555ms/epoch - 15ms/step
Epoch 2/100
36/36 - 0s - loss: 0.5227 - accuracy: 0.7491 - val_loss: 0.4946 - val_accuracy: 0.7248 - 68ms/epoch - 2ms/step
Epoch 3/100
36/36 - 0s - loss: 0.4577 - accuracy: 0.7664 - val_loss: 0.4596 - val_accuracy: 0.7450 - 71ms/epoch - 2ms/step
Epoch 4/100
36/36 - 0s - loss: 0.4245 - accuracy: 0.7818 - val_loss: 0.4444 - val_accuracy: 0.7450 - 69ms/epoch - 2ms/step
Epoch 5/100
36/36 - 0s - loss: 0.4045 - accuracy: 0.7958 - val_loss: 0.4334 - val_accuracy: 0.7494 - 68ms/epoch - 2ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3924 - accuracy: 0.8008 - val_loss: 0.4309 - val_accuracy: 0.7450 - 67ms/epoch - 2ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3837 - accuracy: 0.7980 - val_loss: 0.4160 - val_accuracy: 0.7629 - 71ms/epoch - 2ms/step
Epoch 8/100
36/36 - 0s - loss: 0.3778 - accuracy: 0.8019 - val_loss: 0.4226 - val_accuracy: 0.7651 - 69ms/epoch - 2ms/step
Epoch 9/100
36

#### It seems that after 9-12 epochs, callback is stopping the training, pure sign of overfitting

### Testing the model

This is the final phase of our project: We will TEST our model on the data it has NEVER seen before. The results we get indicates how well our model will perform in real life.

In [20]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 1ms/step - loss: 0.4114 - accuracy: 0.7723


In [26]:
# Printing Out the Final Results.
print(f'Test loss:{test_loss:.2}, Test Accuracy:{(test_accuracy*100):.4}%')

Test loss:0.41, Test Accuracy:77.23%
